# Задание 1

In [ ]:
import sys
sys.path.append('/code')

from os import environ
import psycopg2
import pandas as pd

from task_1_generate_data import generate_data

In [ ]:
DB_HOST = environ["DB_HOST" ]
DATA_DIR = environ["TASK_1_DATA_DIR"]

In [ ]:
sql_uri = f"postgresql://postgres:postgres@{DB_HOST}:5432/task_1"

In [ ]:
# Вспомогательная функция
def execute_df(query):
    pg_cursor = psycopg2.connect(sql_uri).cursor()
    pg_cursor.execute(query)
    names = [metadata[0] for metadata in pg_cursor.description]
    return pd.DataFrame.from_records(pg_cursor.fetchall(), columns=names)

## Подготовка данных

В задаче указаны разные базы для разных таблиц но тогда задачу не решить написав один SQL на задание. Поэтому я все таблицы сложил в одну базу.

In [ ]:
# Эта команда:
#   Создаст базу данных вместе с таблицами
#   сгенерирует CSV с фейковыми данными и загрузит в базу

generate_data(users=1000,
              logins_up_to_per_user=3,
              operations_up_to_per_login=10,
              orders_up_to_per_login=100)

### Задача 1

Идея запроса в том, чтобы отследить время перехода между этапами воронки в рамках каждого логина. На здравом смысле кажется что воронки должны считаться по логинам. Но так как нам нужно считать среднее время для клиента, то после расчёта времени перехода между этапами воронки выбирается миинимальное время для каждого этапа в рамках клиента (user_uid). То есть если у клиента есть два логина и в время перехода между этапами вопронки выглядит как: [1, 3]; [3; 2] то время перехода между этапами для клиента будет выбрано как: [1, 2].

In [ ]:
query = """
with user_logins as (
    select uid,
           u.registration_date,
           country,
           login
    from tb_users as u
             left join
         tb_logins as l
         on
                 u.uid = l.user_uid
                 and account_type = 'real'
    where u.registration_date >= current_timestamp - interval '90 days'
),


     logins_step1 as (
         select uid,
                u.login,
                country,
                registration_date,
                first_operation_date                       as first_operation_date,
                (first_operation_date - registration_date) as time_to_step1
         from user_logins as u
                  left join (
             select login,
                    min(operation_date) as first_operation_date
             from tb_operations
             where operation_type = 'deposit'
             group by login
         )
             as op
                            on op.login = u.login
         where op.login is not null
           and op.first_operation_date is not null
     ),

     logins_step2 as (
         select uid,
                u.login,
                country,
                first_operation_date,
                first_order_date,
                time_to_step1,
                (first_order_date - first_operation_date) as time_to_step2
         from logins_step1 as u
                  left join (
             select login,
                    min(order_close_date) as first_order_date
             from tb_orders
             group by login
         ) as ord
                            on ord.login = u.login
         where ord.login is not null
           and ord.first_order_date is not null
     )

select user_logins.country,
       count(user_logins.uid) as users_step0,
       count(users_step1.uid) as users_step1,
       count(users_step2.uid) as users_step2,
       avg(time_to_step1)     as avg_time_to_step1,
       avg(time_to_step2)     as avg_time_to_step2
from user_logins
         left join (
    select uid,
           min(time_to_step1) as time_to_step1
    from logins_step1
    group by uid
) as users_step1
                   on users_step1.uid = user_logins.uid
         left join (
    select uid,
           min(time_to_step2) as time_to_step2
    from logins_step2
    group by uid
) as users_step2
                   on users_step1.uid = users_step2.uid
group by country
order by users_step0 desc;
"""
df = execute_df(query)
df

### Задача 2

In [ ]:
query = """
select country,
       count(uid) as users_count,
       sum(
               case
                   when avg_amount >= 1000 then 1
                   else 0
                   end
           )      as users_with_avg_deposit_gt_1000
from (
         select country, uid, avg(amount) as avg_amount
         from tb_users as u
                  left join tb_logins as l on l.user_uid = u.uid
                  left join tb_operations t on l.login = t.login
         where operation_type = 'deposit'
         group by country, uid
     ) as a1
group by country
order by users_count desc;
"""
df = execute_df(query)
df

### Задача 3

In [ ]:
query = """
with ordered_deposits as (
    select uid,
           l.login,
           operation_date,
           row_number() over (partition by uid order by operation_date) as deposit_order
    from tb_users as u
             left join tb_logins as l on l.user_uid = u.uid
             left join tb_operations op on l.login = op.login
    where operation_type = 'deposit'
)
select *
from ordered_deposits
where deposit_order < 4;
"""
df = execute_df(query)
df